In [ ]:
#r "nuget: Microsoft.DotNet.Cli.Utils"

open Microsoft.DotNet.Cli.Utils

let runDotNetBuild() =
    let result = 
        Command
            .CreateDotNet("build", [])
            .CaptureStdOut()
            .CaptureStdErr()
            .Execute()

    let output = result.StdOut
    let error = result.StdErr
    let exitCode = result.ExitCode

    (output, error, exitCode)

// Usage
let (output, error, exitCode) = runDotNetBuild()

printfn "Exit Code: %d" exitCode
printfn "Output:\n%s" output
printfn "Error:\n%s" error

In [ ]:
let buildDir = "./bin/Debug/net8.0" 
let copyDir = "./bin/Debug/notebook"

// Clear existing copy directory
if System.IO.Directory.Exists(copyDir) then 
    System.IO.Directory.Delete(copyDir, true)

let rec copyBuildDir (source : string) (target: string) =
    System.IO.Directory.CreateDirectory(target) |> ignore  
    for file in System.IO.Directory.GetFiles(source) do 
        File.Copy(file, System.IO.Path.Combine(target, System.IO.Path.GetFileName(file)))

    for dir in System.IO.Directory.GetDirectories(source) do
        let newDir = System.IO.Path.Combine(target, System.IO.Path.GetFileName(dir)) 
        copyBuildDir dir newDir

copyBuildDir buildDir copyDir

In [ ]:
#r "./bin/Debug/notebook/studies.dll"
#r "./bin/Debug/notebook/core.fs.dll"

#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: MathNet.Numerics"

In [ ]:
// read params from files

let resultsDirectory = System.IO.File.ReadAllText("results_directory.txt")
let studiesDirectory = System.IO.File.ReadAllText("study_directory.txt")
let inputFilename    = $"{studiesDirectory}\\signals.csv"

System.Console.WriteLine("Study directory: " + studiesDirectory)
System.Console.WriteLine("Input filename: " + inputFilename)
System.Console.WriteLine("Results directory: " + resultsDirectory)

In [ ]:
open FSharp.Data
open Plotly.NET
open core.fs.Stocks

let outputToConsole (message:string) =
    System.Console.WriteLine(message)
        
// functions that deal with outcomes and turn them into summaries, describes them, etc
let columnChart label keys values =
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
let histogramChart label bins values =
    let chart = 
        Chart.Histogram(
            X = values,
            NBinsX = bins,
            Name = label
        )
    chart.Display() |> ignore

let outputSummary outputFunc includeDistributionChart (summary:studies.Trading.TradeSummary) = 

    // this ensures that the numbers align instead of negative EV taking up more space
    let evPositiveSign =
        match summary.EV with
        | x when x >= 0m -> "+"
        | _ -> ""
    
    let format = "0.00"
    let percentFormat = "00.00%"
    outputFunc $"{evPositiveSign}{summary.EV.ToString(percentFormat)} EV -> {summary.WinPct.ToString(percentFormat)} \t {summary.AvgWin.ToString(percentFormat)} \t {summary.AvgLoss.ToString(percentFormat)} \t {summary.AvgGainLoss.ToString(format)} avg gain/loss \t {summary.TotalGain.ToString(format)} total gain  \t [{summary.NumberOfTrades} trades] \t {summary.StrategyName}"
        
    match includeDistributionChart with
    | true ->
        histogramChart "Gain Distribution" 40 (summary.Gains)
    | false -> ()
    
    
// filter out outcomes that have minimum of trades we are interested in
let minimumTradesPerCategory = 10

let createTradeSummaries outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Trading.TradeOutcome) -> t.Strategy) // all strategies, no filters

    tradesGroupedByStrategy
    |> Seq.map (
        fun (strategy, trades) ->
            let name = $"{strategy}"
            let summary = trades |> studies.Trading.TradeSummary.create name
            match summary.NumberOfTrades with
            | x when x < minimumTradesPerCategory -> None
            | _ -> Some summary
    )
    |> Seq.choose id
    

type OutcomeGrouping =
    | Signal
    | Strategy
    
let describeOutcomes outputFileOption includeEVChart includeDistributionChart tradeSummaries =
    
    let outputContent message =
        match outputFileOption with
        | None -> ()
        | Some outputFile -> System.IO.File.AppendAllText(outputFile, message + System.Environment.NewLine)
        System.Console.WriteLine(message)
        
    let groupedData = tradeSummaries |> Seq.groupBy (fun (o:studies.Trading.TradeSummary) -> o.StrategyName)
    
    groupedData
    |> Seq.iter (
        fun (groupingName,summaries) ->
            
            outputContent $"{groupingName}"
            
            let sortedSummaries = summaries |> Seq.sortByDescending (fun summary -> summary.EV)
            
            sortedSummaries |> Seq.iter (fun summary -> outputSummary outputContent includeDistributionChart summary)
            
            if includeEVChart then
                let keys = sortedSummaries |> Seq.map (fun summary -> summary.StrategyName)
                let values = sortedSummaries |> Seq.map (fun summary -> summary.EV)
            
                columnChart groupingName keys values
                
            outputContent ""
            outputContent ""
    )
    
let highlightStrategies tradeSummaries =
        
    let sortFunctions = [
        ("EV", fun (t:studies.Trading.TradeSummary) -> t.EV)
        //("Total Gain", fun (t:studies.ScreenerStudy.TradeSummary) -> t.TotalGain)
        //("Win %", fun (t:studies.ScreenerStudy.TradeSummary) -> t.WinPct)
        //("Avg Gain", fun (t:studies.ScreenerStudy.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Strategies sorted by: {sortLabel}"
        
        let summaryFunc = outputSummary outputToConsole false
        let descending =
            tradeSummaries
            |> Seq.sortByDescending sortFunction
            |> Seq.iter summaryFunc
            
        printfn ""
        printfn ""
    )
    
let saveTradeOutcomesToCsv resultsDirectory tradeSummaries =
    tradeSummaries
    |> Seq.iter(fun (summary:studies.Trading.TradeSummary) ->
        let invalidCharacters = System.IO.Path.GetInvalidFileNameChars()
        let cleanedFilename = invalidCharacters |> Seq.fold (fun (n:string) (c:char) -> n.Replace(c, '_')) summary.StrategyName
        let finalFilename = $"{cleanedFilename}.csv"
        let rows = studies.BreakoutStudy.createTradeOutcomesOutput summary.Trades
        let finalPath = System.IO.Path.Combine(resultsDirectory,finalFilename)
        rows.Save(finalPath)
    )        

In [ ]:
let inputSignals =
    inputFilename
    |> studies.BreakoutStudy.BreakoutSignalRecord.Load
    |> _.Rows
    |> Seq.map studies.BreakoutStudy.BreakoutSignalRecordWrapper
    |> List.ofSeq
    
outputToConsole $"Loaded {inputSignals.Length} signals"


In [ ]:
open MathNet.Numerics

// analyze data first
// visualize current slopes
let createHistogram label buckets (func:studies.BreakoutStudy.BreakoutSignalRecordWrapper->decimal) (filter:decimal->bool) (records:studies.BreakoutStudy.BreakoutSignalRecordWrapper list) =
    let slopes = records |> List.map func |> List.filter filter
    
    histogramChart label buckets slopes
    
let showStatistics label (func:studies.BreakoutStudy.BreakoutSignalRecordWrapper->float) (records:studies.BreakoutStudy.BreakoutSignalRecordWrapper list) =
    let values = records |> List.map func
    let stats = MathNet.Numerics.Statistics.DescriptiveStatistics(values)
    let mean = stats.Mean
    let stdDev = stats.StandardDeviation
    $"{label}: count: {records.Length}, mean: {mean}, stdDev: {stdDev}" |> outputToConsole
    stats
    
  
let priceSlopeStats = showStatistics "Price Slope" (fun s -> s.Row.PriceSlope |> float) inputSignals
createHistogram "Price Slope" 1000 (fun s -> s.Row.PriceSlope) (fun slope -> slope > -10m && slope < 10m) inputSignals

let priceAngleStats = showStatistics "Price Angle" (fun s -> s.Row.PriceDegrees |> float) inputSignals
createHistogram "Price Degrees" 10 (fun s -> s.Row.PriceDegrees) (fun s -> true) inputSignals

let volumeSlopeStats = showStatistics "Volume Slope" (fun s -> s.Row.VolumeSlope |> float) inputSignals
createHistogram "Volume Slope" 1000 (fun s -> s.Row.VolumeSlope) (fun volume -> true) inputSignals

let volumeAngleStats = showStatistics "Volume Angle" (fun s -> s.Row.VolumeDegrees |> float) inputSignals
createHistogram "Volume Degrees" 10 (fun s -> s.Row.VolumeDegrees) (fun s -> true) inputSignals

let volumeRateStats = showStatistics "Volume Rate" (fun s -> s.Row.BreakoutVolumeRate |> float) inputSignals
createHistogram "Volume Rate" 100 (fun s -> s.Row.BreakoutVolumeRate) (fun v -> true) inputSignals

// let signalsOutsidePriceSlope = inputSignals |> List.filter (fun s -> s.Row.PriceSlope > decimal (priceSlopeStats.Mean + priceSlopeStats.StandardDeviation * 1.5) * 1m)
// signalsOutsidePriceSlope |> List.sortBy (fun s -> s.Row.PriceSlope) |> List.iter(fun s -> $"{s.Row.Ticker} - {s.Row.Date} - {s.Row.PriceSlope}" |> outputToConsole)

In [ ]:

// this step is for any additional filtering, but right now not doing
// any filtering, and only casting to ISignal that Trading module understands
let signals = 
    inputSignals
    |> Seq.cast<studies.DataHelpers.ISignal>
    |> Seq.toList

outputToConsole $"After filtering, we have {signals.Length} signals"

let priceFunc ticker = studies.DataHelpers.getPricesFromCsv studiesDirectory ticker

let verbosityOn = false

let strategies = [
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None None   // no stop, buy and hold or sell and hold
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 5) None // no stop, but only hold for 5 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 5) None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 10) None // no stop, but only hold for 10 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 10) None    
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 30) None // no stop, but only hold for 30 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 30) None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 60) None // no stop, but only hold for 60 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 60) None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None (Some 0.05m) // 5% stop
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None (Some 0.1m) // 10% stop
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None (Some 0.1m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None (Some 0.2m) // 20% stop
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None (Some 0.2m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 30) (Some 0.05m) // 5% stop plus holding at most 30 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 30) (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 60) (Some 0.05m) // 5% stop plus holding at most 60 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 60) (Some 0.05m)
    studies.Trading.strategyWithSignalOpenAsStop verbosityOn  // signal open as stop
    studies.Trading.strategyWithSignalCloseAsStop verbosityOn // signal close as stop
    studies.Trading.strategyWithTrailingStop verbosityOn StockPositionType.Long 0.1m  // trailing stop of 10%
    studies.Trading.strategyWithTrailingStop verbosityOn StockPositionType.Short 0.1m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Long 0.05m 0.1m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Short 0.05m 0.1m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Long 0.1m 0.2m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Short 0.1m 0.2m
]

let outcomes = 
    strategies
    |> studies.Trading.runTrades priceFunc signals
    |> Async.RunSynchronously
    |> Seq.toList

printfn "Finished running trades, generated:"
printfn $"{outcomes.Length} trade outcomes"

In [ ]:
// leave this cell for outcome checking from time to time when troubleshooting
//outcomes |> List.filter (fun (o:studies.ScreenerStudy.TradeOutcomeOutput.Row) -> o.Screenerid = 28 && o.OpenPrice < o.Sma200 && o.Strategy = "Buy")

In [ ]:
printfn "Generating trade summaries..."
let tradeSummaries = outcomes |> createTradeSummaries |> Seq.toList
printfn $"{tradeSummaries.Length} summaries generated"

printfn "Saving trade summaries..."
tradeSummaries |> saveTradeOutcomesToCsv resultsDirectory

In [ ]:
"Highlighting trades" |> outputToConsole

tradeSummaries |> highlightStrategies

In [ ]:
// tradeSummaries
// |> describeOutcomes None false false
//
// printfn "Finished"

In [ ]:
let exploreCertainStrategies() =
    let strategiesOfInterest = [
        "SL of"
        "as stop"
    ]

    let summarizeSubset (name:string) =
        tradeSummaries
            |> Seq.filter (fun summary -> summary.StrategyName.Contains(name))
            |> Seq.sortByDescending (fun summary -> summary.EV)
            |> Seq.iter (fun summary -> 
                outputSummary outputToConsole false summary
            )

        printfn ""

    strategiesOfInterest |> List.iter (fun n -> summarizeSubset n)

// exploreCertainStrategies()

In [ ]:
// sanity check, print random trades, look over them, investigate specific trades if needed
let positionDescription (signal:studies.Trading.TradeOutcome) =
    // it will consist of components that describe signal
    // gap up or gap down
    // price>20, price<20, put together into a string

    let signalDescription =
        match signal.Signal.Screenerid |> Option.defaultValue 0 with
        | 28 -> "New High"
        | 29 -> "Top Gainer"
        | 30 -> "Top Loser"
        | 31 -> "New Low"
        | _ -> "Generic Signal"
    
    String.concat " " [signalDescription]
        
let outcomeDescription (x:studies.Trading.TradeOutcome) =
    let gain = x.PercentGain.ToString("##.##%")
    let signal = x |> positionDescription
    printfn $"{x.Ticker} \t {x.Strategy} \t Profit: {gain} \t  {x.Opened:d} -> {x.Closed:d} \t {x.OpenPrice} -> {x.ClosePrice}. VA: {signal}, strat: {x.Strategy}"
    
let random = System.Random()

let tradeStrategyOfInterest = "Buy"

//let filterFunc x = x |> signalDescription |> _.Contains(tradeStrategyOfInterest)
let filterFunc (x:studies.Trading.TradeOutcome) = x.Strategy = tradeStrategyOfInterest

outcomes
|> Seq.filter filterFunc
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription

printfn ""
printfn $"Max gain trade from {tradeStrategyOfInterest}"

outcomes
|> Seq.filter filterFunc
|> Seq.sortByDescending (fun x -> x.PercentGain)
|> Seq.truncate 10
|> Seq.iter outcomeDescription

printfn ""
printfn "Max gain overall"

outcomes
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

In [ ]:
Plotly.NET.Defaults.DefaultWidth <- 900

let metrics = [
    ("EV", fun (s:studies.Trading.TradeSummary) -> s.EV)
    ("Win/Loss", fun s -> s.AvgGainLoss)
    ("Win %", fun s -> s.WinPct)
    ("Avg Win", fun s -> s.AvgWin)
    ("Avg Loss", fun s -> s.AvgLoss)
    ("Number of Trades", fun s -> s.NumberOfTrades)
]

metrics
|> List.iter( fun (label,metricFunc) ->

    outputToConsole $"sorting for {label}..."
    
    let sorted = tradeSummaries |> Seq.sortByDescending metricFunc |> Seq.toList
    
    let values = sorted |> List.map metricFunc
    let keys = sorted |> List.map (fun summary -> summary.StrategyName)

    outputToConsole $"mapped keys and values for {label}, charting..."
    
    // values |> List.zip keys |> List.iter (fun (l,v) -> outputToConsole $"{l}: {v}")
    
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
    outputToConsole ""
)

In [ ]:
// let's take only the trades where EV is positive
outputToConsole "Positive EVs only, sorted by EV descending"

tradeSummaries
|> List.filter (fun s -> s.EV > 0m)
|> List.sortByDescending (fun s -> s.EV)
|> List.iter (fun s -> outputSummary outputToConsole false s)

In [ ]:
let benchmarkStrategyName = "Sell"

//tradeSummaries |> List.map _.StrategyName |> List.distinct |> List.iter outputToConsole
let benchmarkStrategy = tradeSummaries |> List.find (fun s -> s.StrategyName = benchmarkStrategyName)

outputSummary outputToConsole false benchmarkStrategy

In [ ]:
//let newHighsWithPriceBelow20 = outcomes |> Seq.filter (fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid |> Option.defaultValue 0 = 28 && o.OpenPrice < (o.Sma20 |> Option.defaultValue 0m) )

//let count = newHighsWithPriceBelow20 |> Seq.length

//printfn "Found %i" count
//newHighsWithPriceBelow20 |> Seq.take 10 |> Seq.iter outcomeDescription

In [ ]:
//let topLoserPriceAbove20 = outcomes |> Seq.filter (fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid |> Option.defaultValue 0 = 30 && o.OpenPrice > (o.Sma20 |> Option.defaultValue 0m) && o.Strategy = "Sell")

//let count = topLoserPriceAbove20 |> Seq.length

//printfn "Found %i" count
//topLoserPriceAbove20 |> Seq.take 30 |> Seq.iter outcomeDescription

In [ ]:
let summarizeRecords label (outcomes:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row seq) = 
    let numberOfTrades = outcomes |> Seq.length
    
    if numberOfTrades = 0 then
        System.Console.WriteLine($"{label} produced no trades")
    else
        let winners = outcomes |> Seq.filter (fun o -> o.PercentGain > 0m)
        let losers = outcomes |> Seq.filter (fun o -> o.PercentGain < 0m)
        let numberOfWinners = winners |> Seq.length
        let numberOfLosers = losers |> Seq.length
        let win_pct = decimal numberOfWinners / decimal numberOfTrades
        let avg_win =
            match numberOfWinners with
            | 0 -> 0m
            | _ -> winners |> Seq.averageBy (_.PercentGain)
        let avg_loss =
            match numberOfLosers with
            | 0 -> 0m
            | _ -> losers |> Seq.averageBy (_.PercentGain)
        let avg_gain_loss =
            match avg_loss with
            | 0m -> 0m
            | _ -> avg_win / avg_loss |> Math.Abs
        let ev = win_pct * avg_win - (1m - win_pct) * (avg_loss |> Math.Abs)
        let totalGain = outcomes |> Seq.sumBy (_.PercentGain)

        System.Console.WriteLine($"EV : {ev:P}\t Win: {win_pct:P}, Avg Win: {avg_win:P}, Avg Loss: {avg_loss:P}, total: {numberOfTrades}, {label}")

In [ ]:
let load filename =
    let filepath = @$"{resultsDirectory}\{filename}"
    studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Load(filepath)
    
let summarize (dataSetName:string) (records:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row seq) =

    System.Console.WriteLine(dataSetName)
    
    summarizeRecords $"All" records

    // let's take only those that have volume rate above 
    records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 2m && r.BreakoutVolumeRate < 3m) |> summarizeRecords "Breakout 2-3"
    records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 3m && r.BreakoutVolumeRate < 4m) |> summarizeRecords "Breakout 3-4"
    records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 4m && r.BreakoutVolumeRate < 5m) |> summarizeRecords "Breakout 4-5"
    records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 5m) |> summarizeRecords "Breakout 5+"
    
    // let's see what's the difference between rising and falling price slope
    records |> Seq.filter (fun r -> r.PriceSlope < 0m) |> summarizeRecords "Negative Price Slope"
    records |> Seq.filter (fun r -> r.PriceSlope > 0m) |> summarizeRecords "Positive Price Slope"
    
    // let's see what the angle breakouts look like
    //records |> Seq.filter (fun r -> r.PriceDegrees <= -40m) |> summarizeRecords "Price deg < -40"
    //records |> Seq.filter (fun r -> r.PriceDegrees < -20m && r.PriceDegrees > -40m) |> summarizeRecords "Price deg -40 to -20"
    //records |> Seq.filter (fun r -> r.PriceDegrees < 20m  && r.PriceDegrees >= -20m) |> summarizeRecords "Price deg -20 to 20"
    //records |> Seq.filter (fun r -> r.PriceDegrees < 40m && r.PriceDegrees >= 20m) |> summarizeRecords "Price deg 20 to 40"
    //records |> Seq.filter (fun r -> r.PriceDegrees >= 40m) |> summarizeRecords "Price deg > 40"
    
    System.Console.WriteLine()
        

In [ ]:
let nofilter rows = rows
let above30PriceFilter rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.OpenPrice >= 30m) rows
let industryCycleUp rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.IndustryCycle = "Up") rows
let industryCycleDown rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.IndustryCycle = "Down") rows
let spyShortTermUp rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.SpyShortTermCycle = "Up") rows
let spyShortTermDown rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.SpyShortTermCycle = "Down") rows
let spyLongTermUp rows =  Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.SpyLongTermCycle = "Up") rows
let spyLongTermDown rows =  Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.SpyLongTermCycle = "Down") rows

let filters = [
    "None", nofilter
    "Price above 30", above30PriceFilter
    "Industry Cycle Up", industryCycleUp
    "Industry Cycle Down", industryCycleDown
    "Spy Short Term Up", spyShortTermUp
    "Spy Short Term Down", spyShortTermDown
    "Spy Long Term Up", spyLongTermUp
    "Spy Long Term Down", spyLongTermDown    
]

filters
|> List.map(fun (filterName, filterFunc) ->
    
    $"************ Filter applied: {filterName}" |> outputToConsole
    
    tradeSummaries
    |> List.map (fun s -> s.StrategyName)
    |> List.iter (fun strategyName ->
        let csv = load $"{strategyName}.csv"
        csv.Rows |> filterFunc |> summarize strategyName 
    )
)


In [ ]:
#r "nuget: XPlot.Plotly"

open XPlot.Plotly

let csv = load "Buy.csv"

let degrees = csv.Rows |> Seq.map (fun s -> s.VolumeDegrees)
let gains = csv.Rows |> Seq.map (fun s -> s.PercentGain * 100m)

// Create a scatter plot
let trace =
    Scatter(
        x = degrees,
        y = gains,
        mode = "markers",
        marker = Marker(size = 10)
    )

// Set up the layout
let layout =
    Layout(
        title = "Price Degrees vs Gains",
        xaxis = Xaxis(title = "Price Degrees"),
        yaxis = Yaxis(title = "Gains")
    )

// Create and display the chart
[trace]
|> Chart.Plot
|> Chart.WithLayout layout
|> Chart.Show

// now histogram of gains
let histogramTrace =
    Histogram(
        x = gains,
        name = "Gains Distribution",
        marker = Marker(color = "rgba(255, 100, 102, 0.7)")
    )

[histogramTrace]
|> Chart.Plot
|> Chart.Show